# Permissible Uncertainty in Pressure

This notebook uses data provided by Dow for the solubility of CO2 in polyol (VORANOL 360) and known uncertainties in the equipment used for Parr-reactor solubility measurements to estimate the permissible uncertainty in the pressure measurement. This estimate will allow us to select an appropriate pressure transducer.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# USER PARAMETERS

# PERMISSIBLE UNCERTAINTIES
# max permissible fractional uncertainty in solubility
sigma_sol_frac_max = 0.1
# uncertainty in volume of polyol [mL]
sigma_V_poly = 3
# uncertainty in initial mass of dry ice [g]
sigma_m0 = 0.1

# VARIABLES
# volume of polyol [mL]
V_poly = 150

# FIXED UNCERTAINTY
# uncertainty in temperature [C]
sigma_T = 1

# CONSTANTS
# universal gas constant (J/mol.K)
R = 8.314
# pascals per psi
psi2pa = 6894.75
# m^3 per mL
mL2m3 = 1E-6
# molar mass of carbon dioxide [g/mol]
mw_co2 = 44
# density of glycerol [g/mL]
rho_poly = 1.08
# volume of Parr reactor [mL]
V_tot = 233

Load data for solubility of CO2 in polyol at 25 C (298 K)

In [2]:
# copy-paste data from file "co2_solubility_pressures.xlsx"
data = np.array([[198.1, 0.0372],
        [405.6, 0.0821],
        [606.1, 0.1351],
        [806.8, 0.1993],
        [893.9, 0.2336]])
# temperature [K]
T = 298

# first column is pressure in psia
p_psia = data[:,0]
# second column is solubility in fraction w/w
solubility = data[:,1]

Convert data into the quantities needed to compute fractional uncertainty.

In [3]:
# convert pressure to Pa
p_pa = psi2pa * p_psia

# compute volume of gas phase in m^3
V_gas = V_tot - V_poly
V_gas_m3 = mL2m3 * V_gas

# number of moles in gas phase
n_co2_gas = p_pa*V_gas_m3/(R*T)
# mass of co2 in the gas phase [g]
m_co2_gas = mw_co2 * n_co2_gas

# compute mass of co2 in solution [g]
m_poly = rho_poly * V_poly
m_co2_soln = solubility * m_poly

Compute maximum permissible fractional uncertainty

In [4]:
term1 = (m_co2_soln**2*(sigma_sol_frac_max**2 - (sigma_V_poly/V_poly)**2) - sigma_m0**2)/m_co2_gas**2

sigma_P_frac2 = term1 - (sigma_V_poly/V_poly)**2 - (sigma_T/T)**2

print(m_co2_soln**2*(sigma_sol_frac_max**2))
print(m_co2_soln**2*(sigma_V_poly/V_poly)**2)
print(sigma_m0**2)
print(sigma_P_frac2)

[ 0.36317497  1.7689532   4.7900575  10.4242454  14.32107786]
[0.014527   0.07075813 0.1916023  0.41696982 0.57284311]
0.010000000000000002
[0.08313625 0.09894168 0.12051814 0.14828631 0.16604765]


In [5]:
print(np.sqrt(sigma_P_frac2)*p_pa/1E5)

[ 3.93820403  8.79642302 14.50738122 21.42076317 25.11445884]


With 35 mL of polyol per batch, the uncertainty in the pressure transducer just needs to be 1% of the measured pressure. That might be too low since the most expensive pressure transducers offer 0.25% of the maximum (which will need to be at least 100 bar, maybe 200 bar to go up to 100 bar >= 0.25 bar, which is only 1% of 25 bar or greater).

With 60 mL of polyol per batch, the uncertainty in the pressure transducer just needs to be 5% of the measured pressure. If we consider a minimum pressure of 10 bar, that's 0.5 bar, which may still be really expensive.

Most surprisingly, I can get the same precision in measuring the solubility with **12%** error in the pressure using 100 mL of polyol and 2 mL uncertainty as with **2%** error in the pressure using 40 mL of polyol and 0.5 mL uncertainty. Clearly filling up the cup is the more effective method for improving precision.

### Conclusion

All of this is to say that the mass of CO2 dissolved in solution is the dominant term in determining the uncertainty. This term increases linearly with pressure and volume of polyol.

In [73]:
.17/1.43

0.1188811188811189